In [33]:
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
import os
import sys
from pandas.plotting import table


# Définir le chemin vers le dossier 'algo_python'
BASE_DIR = os.path.abspath(os.path.join(os.getcwd(), 'algo_python'))
sys.path.append(BASE_DIR)

# Importer les fonctions d'entraînement
from algo_python.knn_model import train_knn_model
from algo_python.random_forest_model import train_random_forest_model
from algo_python.regression_logistique_model import train_regression_model

# Charger les données depuis le dossier 'données'
csv_path = os.path.join(os.getcwd(), 'données', 'donnée_malades_cardiaques.csv')
df = pd.read_csv(csv_path, index_col=0)

In [34]:

X = df.drop(columns=['Maladie_cardiaque'])  
Y = df['Maladie_cardiaque']

# Entraînement (retourne aussi le scaler)
rl, scaler = train_regression_model(X, Y)  # Utiliser le même scaler

# Prendre un échantillon de test
num_samples = 100
X_sample = X.sample(n=num_samples, random_state=42)
Y_sample = Y.loc[X_sample.index]

# Appliquer la **même normalisation** qu'à l'entraînement
X_sample_scaled = scaler.transform(X_sample)  # Ne pas recréer un scaler !

# Prédictions avec le même modèle et scaler
rl_preds = rl.predict(X_sample_scaled)
rl_probs = rl.predict_proba(X_sample_scaled)
print("Shape de X_train :", X.shape)  # Doit être (N, 16)
print("Shape de X_sample_scaled :", X_sample_scaled.shape)  # Doit être (100, 16)


Shape de X avant la séparation :  (917, 18)
Shape de X_train après séparation :  (733, 18)
Shape de X_test après séparation :  (184, 18)
Shape de X_train_scaled après normalisation :  (733, 18)
Shape de X_test_scaled après normalisation :  (184, 18)
Meilleur modèle trouvé :  LogisticRegression(C=0.1, max_iter=1000, solver='liblinear')
Shape de X_train : (917, 18)
Shape de X_sample_scaled : (100, 18)


In [35]:

def generate_comparison_table(X, Y, knn, rf, rl, scaler, num_samples=100):
    # Sélectionner aléatoirement des échantillons
    X_sample = X.sample(n=num_samples, random_state=42)
    Y_sample = Y.loc[X_sample.index]
    
    # Vérifiez que X_sample a le bon nombre de caractéristiques avant la normalisation
    print(f"Shape de X_sample avant transformation : {X_sample.shape}")

    # Normalisation des données
    X_scaled = scaler.transform(X_sample)
    
    # Vérifiez la forme de X_scaled après transformation
    print(f"Shape de X_scaled après normalisation : {X_scaled.shape}")
    
    # Si le modèle LogisticRegression (rl) a été formé avec 10 caractéristiques, appliquez une sélection
    # ou transformation ici pour que X_scaled ait également 10 caractéristiques avant de faire la prédiction.
    
    # Prédictions des modèles
    knn_preds = knn.predict(X_scaled)
    knn_probs = knn.predict_proba(X_scaled)
    
    rf_preds = rf.predict(X_scaled)
    rf_probs = rf.predict_proba(X_scaled)
    
    
    rl_preds = rl.predict(X_scaled)
    rl_probs = rl.predict_proba(X_scaled)
    
    # Construction du DataFrame de comparaison
    comparison_df = X_sample.copy()
    comparison_df['Vraie_classe'] = Y_sample.values
    
    comparison_df['KNN_Prediction'] = knn_preds
    comparison_df['KNN_Prob_Sain'] = knn_probs[:, 0] * 100
    comparison_df['KNN_Prob_Malade'] = knn_probs[:, 1] * 100
    
    comparison_df['RF_Prediction'] = rf_preds
    comparison_df['RF_Prob_Sain'] = rf_probs[:, 0] * 100
    comparison_df['RF_Prob_Malade'] = rf_probs[:, 1] * 100
    
    comparison_df['RL_Prediction'] = rl_preds
    comparison_df['RL_Prob_Sain'] = rl_probs[:, 0] * 100
    comparison_df['RL_Prob_Malade'] = rl_probs[:, 1] * 100
    # Garder uniquement les colonnes nécessaires
    comparison_df = comparison_df[['Vraie_classe', 
                               'KNN_Prediction', 'KNN_Prob_Sain', 'KNN_Prob_Malade', 
                               'RF_Prediction', 'RF_Prob_Sain', 'RF_Prob_Malade', 
                               'RL_Prediction', 'RL_Prob_Sain', 'RL_Prob_Malade']]

    
    return comparison_df


X = df.drop(columns=['Maladie_cardiaque'])  
Y = df['Maladie_cardiaque']

# Entraînement des modèles
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)
knn = train_knn_model(X, Y)
rf = train_random_forest_model(X, Y)
rl, _ = train_regression_model(X, Y)

# Générer la comparaison
tableau_comparaison = generate_comparison_table(X, Y, knn, rf, rl, scaler)
tableau_comparaison.to_csv('tableau_comparaison.csv', index=False)

# Afficher les premières lignes
tableau_comparaison.head()



Shape de X avant la séparation :  (917, 18)
Shape de X_train après séparation :  (733, 18)
Shape de X_test après séparation :  (184, 18)
Shape de X_train_scaled après normalisation :  (733, 18)
Shape de X_test_scaled après normalisation :  (184, 18)
Meilleur modèle trouvé :  LogisticRegression(C=0.1, max_iter=1000, solver='liblinear')
Shape de X_sample avant transformation : (100, 18)
Shape de X_scaled après normalisation : (100, 18)


c:\Users\user\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(
c:\Users\user\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(


,Vraie_classe,KNN_Prediction,KNN_Prob_Sain,KNN_Prob_Malade,RF_Prediction,RF_Prob_Sain,RF_Prob_Malade,RL_Prediction,RL_Prob_Sain,RL_Prob_Malade
669,0,0,80.327869,19.672131,0,64.0,36.0,0,89.087932,10.912068
30,1,1,47.540984,52.459016,1,42.0,58.0,0,57.108639,42.891361
377,1,1,1.639344,98.360656,1,20.0,80.0,1,4.179884,95.820116
536,1,1,9.836066,90.163934,1,24.0,76.0,1,7.742860,92.257140
807,0,0,100.000000,0.000000,0,92.0,8.0,0,94.199038,5.800962


In [36]:
# Calcul du nombre de bonnes prédictions pour chaque modèle
knn_correct = (tableau_comparaison['KNN_Prediction'] == tableau_comparaison['Vraie_classe']).sum()
rf_correct = (tableau_comparaison['RF_Prediction'] == tableau_comparaison['Vraie_classe']).sum()
rl_correct = (tableau_comparaison['RL_Prediction'] == tableau_comparaison['Vraie_classe']).sum()

# Stocker les résultats dans un dictionnaire
results = {
    'KNN': knn_correct,
    'Random Forest': rf_correct,
    'Régression Logistique': rl_correct
}

# Identifier l'algorithme le plus fiable
best_model = max(results, key=results.get)

# Affichage des résultats
print("\nRésultats de la comparaison :")
for model, correct in results.items():
    print(f"{model} : {correct} bonnes prédictions sur {len(tableau_comparaison)}")

print(f"\n✅ L'algorithme le plus fiable est : {best_model} avec {results[best_model]} bonnes prédictions.")



Résultats de la comparaison :
KNN : 86 bonnes prédictions sur 100
Random Forest : 85 bonnes prédictions sur 100
Régression Logistique : 86 bonnes prédictions sur 100

✅ L'algorithme le plus fiable est : KNN avec 86 bonnes prédictions.


In [37]:

# Choisir combien de lignes tu veux voir (ex: 20)
tableau_preview = tableau_comparaison.head(100)

# Créer une figure
fig, ax = plt.subplots(figsize=(20, 10))  # Ajuste la taille selon ton besoin
ax.axis('off')  # On désactive les axes

# Ajouter le tableau
tbl = table(ax, tableau_preview, loc='center', cellLoc='center', colWidths=[0.1]*len(tableau_preview.columns))

# Style (optionnel)
tbl.auto_set_font_size(False)
tbl.set_fontsize(10)
tbl.scale(1.2, 1.2)  # Ajuste l'échelle si besoin

# Sauvegarder l'image
plt.savefig('tableau_comparaison_image.png', bbox_inches='tight', dpi=300)
plt.close()
